# Extract relevant OSM Features of Germany and convert the cooridate system to EPSG:25832

In [ ]:
#!/bin/bash

!OSM_FILE="data/germany-latest.osm.pbf"
# 2. Bio-Läden
!osmium tags-filter "$OSM_FILE" nwr/shop=greengrocer nwr/shop=health_food nwr/shop=farm -o bio_shops.osm.pbf --overwrite -t_srs EPSG:25832
print ("Fertig: bio_shops.osm.pbf")

# 2. Spielplätze
# Extrahiert Spielplätze (leisure=playground).
print ("Extrahiere Spielplätze...")
!osmium tags-filter "$OSM_FILE" nwr/leisure=playground -o playgrounds.osm.pbf --overwrite -t_srs EPSG:25832
print ("Fertig: playgrounds.osm.pbf")

# 3. Hotels und Hostels
# Extrahiert Hotels (tourism=hotel) und Hostels (tourism=hostel).
print ("Extrahiere Hotels und Hostels...")
!osmium tags-filter "$OSM_FILE" nwr/tourism=hotel nwr/tourism=hostel -o hotels_hostels.osm.pbf --overwrite -t_srs EPSG:25832
print ("Fertig: hotels_hostels.osm.pbf")

# 4. Airbnb-/Ferienwohnungs-Listings
# Extrahiert Ferienwohnungen (tourism=apartment).
print ("Extrahiere Airbnb-/Ferienwohnungs-Listings...")
!osmium tags-filter "$OSM_FILE" nwr/tourism=apartment -o apartments.osm.pbf --overwrite -t_srs EPSG:25832
print ("Fertig: apartments.osm.pbf")

# 5. Gastronomie-Einrichtungen
# Extrahiert Restaurants, Fast Food, Cafés und Bars.
print ("Extrahiere Gastronomie-Einrichtungen...")
!osmium tags-filter "$OSM_FILE" nwr/amenity=restaurant nwr/amenity=fast_food nwr/amenity=cafe nwr/amenity=bar -o gastronomy.osm.pbf --overwrite -t_srs EPSG:25832
print ("Fertig: gastronomy.osm.pbf")

# 6. Schulen und Kindertagesstätten
# Extrahiert Schulen (amenity=school) und Kindergärten (amenity=kindergarten).
print ("Extrahiere Schulen und Kindertagesstätten...")

# Schulen extrahieren
!osmium tags-filter "$OSM_FILE" nwr/amenity=school -o schools.osm.pbf --overwrite -t_srs EPSG:25832
print ("Fertig: schools.osm.pbf")

# Kindergärten extrahieren
!osmium tags-filter "$OSM_FILE" nwr/amenity=kindergarten -o kindergartens.osm.pbf --overwrite -t_srs EPSG:25832
print ("Fertig: kindergartens.osm.pbf")

# 7. Grünflächen/Erholungsgebiete
# Extrahiert Parks (leisure=park) und Wälder (natural=wood).
# Spielplätze (leisure=playground) sind bereits separat extrahiert.
print ("Extrahiere Grünflächen/Erholungsgebiete (Parks, Wälder)...")
!osmium tags-filter "$OSM_FILE" nwr/leisure=park nwr/natural=wood -o green_areas.osm.pbf --overwrite -t_srs EPSG:25832
print ("Fertig: green_areas.osm.pbf")

# 8. Gewässer
# Extrahiert Gewässer (natural=water).
print ("Extrahiere Gewässer...")
!osmium tags-filter "$OSM_FILE" nwr/natural=water -o water_bodies.osm.pbf --overwrite -t_srs EPSG:25832
print ("Fertig: water_bodies.osm.pbf")

# 9. Landnutzungsmix
# Extrahiert Flächen nach Landnutzungstypen (residential, commercial, industrial, farm).
print ("Extrahiere Landnutzungsflächen...")
!osmium tags-filter "$OSM_FILE" nwr/landuse=residential nwr/landuse=commercial nwr/landuse=industrial nwr/landuse=farm -o landuse.osm.pbf --overwrite -t_srs EPSG:25832
print ("Fertig: landuse.osm.pbf")

# 10. Gebäudetypen-Verteilung
# Extrahiert Gebäude nach Typen (residential, commercial, apartments).
print ("Extrahiere Gebäude nach Typen...")
!osmium tags-filter "$OSM_FILE" nwr/building=residential nwr/building=commercial nwr/building=apartments -o building_types.osm.pbf --overwrite -t_srs EPSG:25832
print ("Fertig: building_types.osm.pbf")

# 11. Gebäudedichte (alle Gebäude)
# Extrahiert alle Gebäude (alle Elemente mit building=*).
print ("Extrahiere alle Gebäude für die Gebäudedichte...")
!osmium tags-filter "$OSM_FILE" nwr/building -o all_buildings.osm.pbf --overwrite -t_srs EPSG:25832
print ("Fertig: all_buildings.osm.pbf")

# 12. Durchschnittliche Gebäudekubatur
# Extrahiert Gebäude, die Informationen über Stockwerke oder Höhe haben könnten.
# Die tatsächliche Kubatur muss anschließend mit GIS-Tools berechnet werden.
print ("Extrahiere Gebäude mit Höhen-/Stockwerksangaben...")
!osmium tags-filter "$OSM_FILE" nwr/building --with-tags building:levels,height -o buildings_with_height_levels.osm.pbf --overwrite -t_srs EPSG:25832
print ("Fertig: all_buildings.osm.pbf")
print ("Fertig: buildings_with_height_levels.osm.pbf")

# 13. Dichte an Gesundheitsdienstleistungen
# Extrahiert Arztpraxen und Apotheken.
print ("Extrahiere Gesundheitsdienstleistungen (Ärzte, Apotheken)...")
!osmium tags-filter "$OSM_FILE" nwr/amenity=doctors nwr/amenity=pharmacy -o health_services.osm.pbf --overwrite -t_srs EPSG:25832
print ("Fertig: health_services.osm.pbf")

# 13b. Dichte an Finanzdienstleistungen
# Extrahiert Banken und Geldautomaten.
print ("Extrahiere Finanzdienstleistungen (Banken, Geldautomaten)...")
!osmium tags-filter "$OSM_FILE" nwr/amenity=bank nwr/amenity=atm -o finance_services.osm.pbf --overwrite -t_srs EPSG:25832
print ("Fertig: finance_services.osm.pbf")

# 14. Straßendichte
# Extrahiert alle Wege mit dem Tag highway=*.
print ("Extrahiere alle Straßen für die Straßendichte...")
!osmium tags-filter "$OSM_FILE" w/highway -o all_highways.osm.pbf --overwrite -t_srs EPSG:25832
print ("Fertig: all_highways.osm.pbf")

# 15. Radinfrastruktur-Dichte
# Extrahiert Radwege (highway=cycleway).
print ("Extrahiere Radinfrastruktur...")
!osmium tags-filter "$OSM_FILE" w/highway=cycleway -o cycleways.osm.pbf --overwrite -t_srs EPSG:25832
print ("Fertig: cycleways.osm.pbf")


## Convertierung des osm.pbf zu GeoJson

In [10]:
!ogr2ogr -f GeoJSON data/bio_shops.geojson data/bio_shops.osm.pbf -overwrite -sql "SELECT * FROM points" -t_srs EPSG:25832

## Imports

In [11]:
import geopandas as gpd
import pandas as pd
import os

## Konfiguration

In [37]:
GEMEINDEN_GPKG = "data/DE_VG5000.gpkg"
LAYER = "v_vg5000_gem"
GEMEINDE_ID_COLUMN = "Gemeindeschlüssel_AGS"

DEUTSCHLAND_ATALAS = "data/DeutschlandAtlasData.csv"

EXTRA_FEATURES = "data/bio_shops.geojson"
EXTRA_FEATURES_NAME = "Bio-Läden"


ADD_KM2_DENSITY_COLUMN = False
ADD_POPULATION_DENSITY_COLUMN = False

# --- Überprüfung der Dateipfade ---
if not os.path.exists(GEMEINDEN_GPKG):
    print(f"Fehler: Das GeoPackage der Gemeinden wurde nicht gefunden unter: {GEMEINDEN_GPKG}")
    print("Bitte passen Sie den Pfad 'GEMEINDEN_GPKG' im Skript an.")
    exit()

if not os.path.exists(EXTRA_FEATURES):
    print(f"Fehler: Die GeoJSON-Datei der Bio-Läden wurde nicht gefunden unter: {EXTRA_FEATURES}")
    print("Stellen Sie sicher, dass Sie 'ogr2ogr -f GeoJSON bio_shops.geojson bio_shops.osm.pbf' ausgeführt haben.")
    exit()

# Adding Deutschland Atalas data

In [43]:
da_df = pd.read_csv(DEUTSCHLAND_ATALAS, sep=';', decimal=',')
da_df["ID"] = da_df["ID"].astype(str)

gemeinden_gdf = gemeinden_gdf.merge(da_df, left_on=GEMEINDE_ID_COLUMN, right_on="ID", how="left")
gemeinden_gdf['Name'] = (
        gemeinden_gdf['bev_dicht'] * gemeinden_gdf['Flaeche_km2'])

## Verifikation

In [39]:

try:
    gemeinden_gdf = gpd.read_file(GEMEINDEN_GPKG, layer=LAYER)
    extras_gdf = gpd.read_file(EXTRA_FEATURES)
    
    print(f"Gemeinden geladen. Anzahl der Gemeinden: {len(gemeinden_gdf)}")
    if GEMEINDE_ID_COLUMN not in gemeinden_gdf.columns:
        print(f"Warnung: Die angegebene Gemeinden-ID-Spalte '{GEMEINDE_ID_COLUMN}' wurde nicht gefunden.")
        print(f"Verfügbare Spalten: {gemeinden_gdf.columns.tolist()}")
        print("Bitte korrigieren Sie 'GEMEINDE_ID_COLUMN' im Skript, um Fortzufahren.")
        exit()

except Exception as e:
    print(f"Fehler beim Laden des Gemeinden GeoPackage: {e}")
    exit()

try:
    print(f"Zusätzliche Features geladen. Anzahl der Features: {len(extras_gdf)}")
    # Sicherstellen, dass das CRS übereinstimmt
    if gemeinden_gdf.crs != extras_gdf.crs:
        print(f"CRS-Konflikt! Gemeinden CRS: {gemeinden_gdf.crs}, Zusätzliche Features CRS: {extras_gdf.crs}")
        exit()

except Exception as e:
    print(f"Fehler beim Laden der GeoJSON: {e}")
    exit()


Gemeinden geladen. Anzahl der Gemeinden: 10957
Zusätzliche Features geladen. Anzahl der Features: 7619


In [40]:
joined_data = gpd.sjoin(extras_gdf, gemeinden_gdf, how="inner", predicate="within")

print(f"Räumlicher Join abgeschlossen. Anzahl der gejoiniten Einträge: {len(joined_data)}")

counts = joined_data.groupby(GEMEINDE_ID_COLUMN).size().reset_index(name=EXTRA_FEATURES_NAME)

# Fügen Sie die Zählungen wieder dem ursprünglichen Gemeinden-GeoDataFrame hinzu
gemeinden_gdf = gemeinden_gdf.merge(counts, on=GEMEINDE_ID_COLUMN, how="left")
# Ersetzen Sie NaN-Werte (Gemeinden ohne Läden) durch 0
gemeinden_gdf[EXTRA_FEATURES_NAME] = gemeinden_gdf[EXTRA_FEATURES_NAME].fillna(0).astype(int)

if gemeinden_gdf.crs.is_geographic: # Prüft, ob es ein geografisches CRS ist (wie WGS84 - 4326)
    print("Konvertiere Gemeinden-CRS für genaue Flächenberechnung (z.B. nach EPSG:3035)...")
    gemeinden_gdf_proj = gemeinden_gdf.to_crs(epsg=3035) # ETRS89 / LAEA Europe
    gemeinden_gdf['Flaeche_km2'] = gemeinden_gdf_proj.geometry.area / 1_000_000 # Fläche in km²
    print("Flächenberechnung in km² durchgeführt.")
else:
    print("Gemeinden-CRS ist bereits projiziert. Berechne Fläche in km²...")
    gemeinden_gdf['Flaeche_km2'] = gemeinden_gdf.geometry.area / 1_000_000 # Annahme: CRS-Einheit ist Meter
    print("Flächenberechnung in km² durchgeführt.")


Räumlicher Join abgeschlossen. Anzahl der gejoiniten Einträge: 7591
Gemeinden-CRS ist bereits projiziert. Berechne Fläche in km²...
Flächenberechnung in km² durchgeführt.

--- Beispielergebnisse für die ersten 5 Gemeinden ---
  Gemeindeschlüssel_AGS  Bio-Läden  Flaeche_km2  Dichte_Bio-Läden_pro_km2
0              01001000          8    50.829770                  0.157388
1              01002000          5   112.366512                  0.044497
2              01003000         50   209.136031                  0.239079
3              01004000          1    71.991713                  0.013890
4              01051011          1    49.225731                  0.020315

Die Analyse ist abgeschlossen. Das resultierende GeoDataFrame ist 'gemeinden_gdf'.


In [ ]:
# Liste der zu joinenden GeoJSON-Dateien und deren Spaltennamen
feature_files = [
    ("data/bio_shops.geojson", "Bio_Laeden"),
    ("data/playgrounds.geojson", "Spielplaetze"),
    ("data/hotels_hostels.geojson", "Hotels_Hostels"),
    ("data/gastronomy.geojson", "Gastronomie"),
    ("data/schools_kindergartens.geojson", "Schulen_Kindergärten"),
    ("data/green_areas.geojson", "Gruenflaechen"),
    ("data/water_bodies.geojson", "Gewässer"),
    ("data/landuse.geojson", "Landnutzung"),
    ("data/building_types.geojson", "Gebaeudetypen"),
    ("data/all_buildings.geojson", "Alle_Gebaeude"),
    ("data/buildings_with_height_levels.geojson", "Gebaeude_mit_Hoehenangaben"),
    ("data/health_finance.geojson", "Gesundheit_Finanzen"),
    ("data/all_highways.geojson", "Strassen"),
    ("data/cycleways.geojson", "Radwege")
]

for file_path, feature_name in feature_files:
    if not os.path.exists(file_path):
        print(f"Warnung: Datei {file_path} nicht gefunden. Überspringe {feature_name}.")
        gemeinden_gdf[feature_name] = 0
        continue
    try:
        extras_gdf = gpd.read_file(file_path)
        if gemeinden_gdf.crs != extras_gdf.crs:
            extras_gdf = extras_gdf.to_crs(gemeinden_gdf.crs)
        joined = gpd.sjoin(extras_gdf, gemeinden_gdf, how="inner", predicate="within")
        counts = joined.groupby(GEMEINDE_ID_COLUMN).size().reset_index(name=feature_name)
        gemeinden_gdf = gemeinden_gdf.merge(counts, on=GEMEINDE_ID_COLUMN, how="left")
        gemeinden_gdf[feature_name] = gemeinden_gdf[feature_name].fillna(0).astype(int)
        print(f"{feature_name}: Join und Zählung abgeschlossen.")
    except Exception as e:
        print(f"Fehler beim Join für {feature_name}: {e}")
        gemeinden_gdf[feature_name] = 0


Bio_Laeden: Join und Zählung abgeschlossen.
Warnung: Datei data/playgrounds.geojson nicht gefunden. Überspringe Spielplaetze.
Warnung: Datei data/hotels_hostels.geojson nicht gefunden. Überspringe Hotels_Hostels.
Warnung: Datei data/gastronomy.geojson nicht gefunden. Überspringe Gastronomie.
Warnung: Datei data/schools_kindergartens.geojson nicht gefunden. Überspringe Schulen_Kindergärten.
Warnung: Datei data/green_areas.geojson nicht gefunden. Überspringe Gruenflaechen.
Warnung: Datei data/water_bodies.geojson nicht gefunden. Überspringe Gewässer.
Warnung: Datei data/landuse.geojson nicht gefunden. Überspringe Landnutzung.
Warnung: Datei data/building_types.geojson nicht gefunden. Überspringe Gebaeudetypen.
Warnung: Datei data/all_buildings.geojson nicht gefunden. Überspringe Alle_Gebaeude.
Warnung: Datei data/buildings_with_height_levels.geojson nicht gefunden. Überspringe Gebaeude_mit_Hoehenangaben.
Warnung: Datei data/health_finance.geojson nicht gefunden. Überspringe Gesundheit_Fi

In [42]:
# Dichte pro km² hinzufügen, wenn gewünscht
if ADD_KM2_DENSITY_COLUMN:
    NAME_DENSITY_COLUMN = 'Dichte_' + EXTRA_FEATURES_NAME + '_pro_km2'
    gemeinden_gdf[NAME_DENSITY_COLUMN] = (
        gemeinden_gdf[EXTRA_FEATURES_NAME] / gemeinden_gdf['Flaeche_km2']
    ).replace([float('inf'), -float('inf')], 0).fillna(0) # Unendlich durch 0 ersetzen, NaN durch 0


In [ ]:
# Dichte pro Einwohner hinzufügen, wenn gewünscht
if ADD_POPULATION_DENSITY_COLUMN:
    NAME_DENSITY_COLUMN = 'Dichte_' + EXTRA_FEATURES_NAME + '_pro_Einwohner'
    gemeinden_gdf['Einwohner'] = (
        gemeinden_gdf['bev_dicht'] * gemeinden_gdf['Flaeche_km2'])
    gemeinden_gdf[NAME_DENSITY_COLUMN] = (
        gemeinden_gdf[EXTRA_FEATURES_NAME] / gemeinden_gdf['Einwohner']
    ).replace([float('inf'), -float('inf')], 0).fillna(0) # Unendlich durch 0 ersetzen, NaN durch 0


In [ ]:

# Dichte pro 10.000 Einwohner (falls Bevölkerungsdaten verfügbar sind)

# ANNAHME: Ihr gemeinden.gpkg hat eine Spalte 'Einwohnerzahl'
# Wenn Ihr GeoPackage eine Spalte 'Einwohnerzahl' (oder ähnlich) enthält,
# können Sie diese Zeile entkommentieren und den Spaltennamen anpassen.
# if 'Einwohnerzahl' in gemeinden_gdf.columns:
#     gemeinden_gdf['Dichte_BioLaeden_pro_10000_EW'] = (
#         (gemeinden_gdf[EXTRA_FEATURES_NAME] / gemeinden_gdf['Einwohnerzahl']) * 10000
#     ).replace([float('inf'), -float('inf')], 0).fillna(0)
#     print("Dichte pro 10.000 Einwohner berechnet (falls Einwohnerzahl-Spalte vorhanden).")
# else:
#     print("Keine 'Einwohnerzahl'-Spalte im Gemeinden-GeoPackage gefunden. Dichte pro 10.000 EW nicht berechnet.")


In [ ]:


# --- Ergebnisse anzeigen ---
print("\n--- Beispielergebnisse für die ersten 5 Gemeinden ---")
print(gemeinden_gdf[[GEMEINDE_ID_COLUMN, EXTRA_FEATURES_NAME, 'Flaeche_km2', NAME_KM2_DENSITY_COLUMN]].head())

# Das resultierende GeoDataFrame ist 'gemeinden_gdf'
# Sie können es nun für weitere Analysen oder Visualisierungen verwenden.
print("\nDie Analyse ist abgeschlossen. Das resultierende GeoDataFrame ist 'gemeinden_gdf'.")


In [41]:
gemeinden_gdf.loc[gemeinden_gdf[NAME_KM2_DENSITY_COLUMN] == gemeinden_gdf[NAME_KM2_DENSITY_COLUMN].max()]

,Objektidentifikator,BeginnLebenszeit,Admin_Ebene_ADE,ADE,Geofaktor_GF,GF,Besondere_Gebiete_BSG,BSG,Regionalschlüssel_ARS,Gemeindeschlüssel_AGS,...,Funk_Schlüsselstelle3,FK_S3,Europ_Statistikschlüssel_NUTS,RegioschlüsselAufgefüllt,GemeindeschlüsselAufgefüllt,Wirksamkeit_WSK,geometry,Bio-Läden,Flaeche_km2,Dichte_Bio-Läden_pro_km2
6005,DEBKGVG5000004N5,2019-10-04,Gemeinde,6,mit Struktur,9,Deutschland,1,083365008024,08336024,...,Regierungsbezirk,R,DE139,083365008024,08336024,2009-01-01,"MULTIPOLYGON (((395417.545 5279222.993, 395976...",3,1.906754,1.573355


In [ ]:
# Konvertiere alle extrahierten .osm.pbf-Dateien zu GeoJSON
pbf_geojson_pairs = [
    ("data/bio_shops.osm.pbf", "data/bio_shops.geojson"),
    ("data/playgrounds.osm.pbf", "data/playgrounds.geojson"),
    ("data/hotels_hostels.osm.pbf", "data/hotels_hostels.geojson"),
    ("data/gastronomy.osm.pbf", "data/gastronomy.geojson"),
    ("data/schools_kindergartens.osm.pbf", "data/schools_kindergartens.geojson"),
    ("data/green_areas.osm.pbf", "data/green_areas.geojson"),
    ("data/water_bodies.osm.pbf", "data/water_bodies.geojson"),
    ("data/landuse.osm.pbf", "data/landuse.geojson"),
    ("data/building_types.osm.pbf", "data/building_types.geojson"),
    ("data/all_buildings.osm.pbf", "data/all_buildings.geojson"),
    ("data/buildings_with_height_levels.osm.pbf", "data/buildings_with_height_levels.geojson"),
    ("data/health_finance.osm.pbf", "data/health_finance.geojson"),
    ("data/all_highways.osm.pbf", "data/all_highways.geojson"),
    ("data/cycleways.osm.pbf", "data/cycleways.geojson")
]

for pbf, geojson in pbf_geojson_pairs:
    if not os.path.exists(geojson) and os.path.exists(pbf):
        !ogr2ogr -f GeoJSON {geojson} {pbf} -overwrite -sql "SELECT * FROM points" -t_srs EPSG:25832
        print(f"Konvertiert: {pbf} → {geojson}")
    elif not os.path.exists(pbf):
        print(f"Fehlt: {pbf}")
    else:
        print(f"Bereits vorhanden: {geojson}")
